<a href="https://colab.research.google.com/gist/tkilias/64631716e39bcd523c93f3478019928c/welcome-to-colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyexasol

    100% |████████████████████████████████| 40kB 188kB/s 
    100% |████████████████████████████████| 204kB 2.1MB/s 
    100% |████████████████████████████████| 51kB 1.8MB/s 
    100% |████████████████████████████████| 81kB 1.2MB/s 


In [2]:
EXASOL_DSN="db_container_test:8888"
EXASOL_USER="sys"
EXASOL_PASSWORD="exasol" 
EXASOL_SCHEMA="ML_TEST"

import pyexasol
c = pyexasol.connect(dsn=EXASOL_DSN, user=EXASOL_USER, password=EXASOL_PASSWORD)
c.execute(f"CREATE SCHEMA IF NOT EXISTS {EXASOL_SCHEMA};")
c.open_schema(EXASOL_SCHEMA)
print(c.execute("select 1").fetchall())



[(1,)]


In [3]:
import pandas as pd
c.execute("""
CREATE OR REPLACE TABLE adult (
  age integer,
  workclass varchar(2000000),
  fnlwgt integer,	
  education varchar(2000000),
  education_num integer,	
  marital_status varchar(2000000),
  occupation varchar(2000000),
  relationship varchar(2000000),
  race varchar(2000000),
  sex varchar(2000000),
  capitalgain integer,	
  capitalloss integer,	
  hoursperweek integer,
  native_country varchar(2000000),
  class varchar(2000000)
);
""")
c.execute("""
IMPORT INTO adult 
FROM CSV AT 'https://datahub.io/machine-learning/adult/r/' FILE '1.csv'
SKIP = 1;
""")
print("count:",c.execute("select count(*) from adult").fetchall())
print("raw size in MB",c.execute("select RAW_OBJECT_SIZE/(1024*1024) from EXA_USER_OBJECT_SIZES where OBJECT_NAME='ADULT'").fetchall())
for i in range(5):
    c.execute("""
              insert into adult select * from  adult 
              """)
print("amplified count:",c.execute("select count(*) from adult").fetchall())
print("amplified raw size in MB",c.execute("select RAW_OBJECT_SIZE/(1024*1024) from EXA_USER_OBJECT_SIZES where OBJECT_NAME='ADULT'").fetchall())
c.export_to_pandas("select * from adult limit 10")

count: [(48842,)]
raw size in MB [('5.97541904449462890625',)]
amplified count: [(1562944,)]
amplified raw size in MB [('191.213409423828125',)]


,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITALGAIN,CAPITALLOSS,HOURSPERWEEK,NATIVE_COUNTRY,CLASS
0,4,Private,67841,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,2,United-States,>50K\r
1,2,Local-gov,244522,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,3,3,United-States,>50K\r
2,0,Private,430471,11th,7,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,2,United-States,<=50K\r
3,3,Private,194698,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,3,United-States,<=50K\r
4,1,Private,94235,Bachelors,13,Never-married,Craft-repair,Unmarried,White,Male,0,0,2,United-States,<=50K\r
5,4,Private,188330,HS-grad,9,Married-civ-spouse,Other-service,Husband,White,Male,0,0,4,United-States,<=50K\r
6,3,Local-gov,146181,9th,5,Married-civ-spouse,Transport-moving,Husband,Black,Male,0,0,2,United-States,<=50K\r
7,0,Private,177125,Some-college,10,Never-married,Handlers-cleaners,Not-in-family,White,Male,0,0,0,United-States,<=50K\r
8,1,Self-emp-inc,68330,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,3,United-States,<=50K\r
9,3,Private,95636,Some-college,10,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States,<=50K\r


In [4]:
import textwrap
c.execute("""
CREATE OR REPLACE TABLE adult_with_id AS 
SELECT 
  rownum as id,
  AGE, WORKCLASS, FNLWGT, EDUCATION, EDUCATION_NUM, MARITAL_STATUS, OCCUPATION, 
  RELATIONSHIP, RACE, SEX, CAPITALGAIN, CAPITALLOSS, HOURSPERWEEK, NATIVE_COUNTRY, CLASS
FROM adult;
""")
c.execute(textwrap.dedent("""
CREATE OR REPLACE TABLE WORKCLASS_CATEGORIES AS SELECT rownum - 1 AS id, WORKCLASS 
FROM (SELECT DISTINCT WORKCLASS FROM adult_with_id);
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE TABLE EDUCATION_CATEGORIES AS SELECT rownum - 1 AS id, EDUCATION
FROM (SELECT DISTINCT EDUCATION FROM adult_with_id);
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE TABLE MARITAL_STATUS_CATEGORIES AS SELECT rownum - 1 AS id, MARITAL_STATUS 
FROM (SELECT DISTINCT MARITAL_STATUS FROM adult_with_id);
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE TABLE OCCUPATION_CATEGORIES AS SELECT rownum - 1 AS id, OCCUPATION 
FROM (SELECT DISTINCT OCCUPATION FROM adult_with_id);
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE TABLE RELATIONSHIP_CATEGORIES AS SELECT rownum - 1 AS id, RELATIONSHIP 
FROM (SELECT DISTINCT RELATIONSHIP FROM adult_with_id);
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE TABLE RACE_CATEGORIES AS SELECT rownum - 1 AS id, RACE 
FROM (SELECT DISTINCT RACE FROM adult_with_id);
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE TABLE SEX_CATEGORIES AS SELECT rownum - 1 AS id, SEX 
FROM (SELECT DISTINCT SEX FROM adult_with_id);
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE TABLE NATIVE_COUNTRY_CATEGORIES AS SELECT rownum - 1 AS id, NATIVE_COUNTRY 
FROM (SELECT DISTINCT NATIVE_COUNTRY FROM adult_with_id);
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE TABLE CLASS_CATEGORIES AS SELECT rownum - 1 AS id, CLASS 
FROM (SELECT DISTINCT CLASS FROM adult_with_id);
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE TABLE adult_preprocessed AS 
SELECT
  adult.id,
  1.00 * ("AGE" - MIN("AGE") OVER()) / (MAX("AGE") OVER () - MIN("AGE") OVER ()) AS "AGE", 
  1.00 * ("FNLWGT" - MIN("FNLWGT") OVER()) / (MAX("FNLWGT") OVER () - MIN("FNLWGT") OVER ()) AS "FNLWGT", 
  1.00 * ("EDUCATION_NUM" - MIN("EDUCATION_NUM") OVER()) / (MAX("EDUCATION_NUM") OVER () - MIN("EDUCATION_NUM") OVER ()) AS "EDUCATION_NUM", 
  1.00 * ("CAPITALGAIN" - MIN("CAPITALGAIN") OVER()) / (MAX("CAPITALGAIN") OVER () - MIN("CAPITALGAIN") OVER ()) AS "CAPITALGAIN", 
  1.00 * ("CAPITALLOSS" - MIN("CAPITALLOSS") OVER()) / (MAX("CAPITALLOSS") OVER () - MIN("CAPITALLOSS") OVER ()) AS "CAPITALLOSS", 
  1.00 * ("HOURSPERWEEK" - MIN("HOURSPERWEEK") OVER()) / (MAX("HOURSPERWEEK") OVER () - MIN("HOURSPERWEEK") OVER ()) AS "HOURSPERWEEK", 
  WORKCLASS_CATEGORIES.id AS WORKCLASS, 
  EDUCATION_CATEGORIES.id AS EDUCATION, 
  MARITAL_STATUS_CATEGORIES.id AS MARITAL_STATUS, 
  OCCUPATION_CATEGORIES.id AS OCCUPATION, 
  RELATIONSHIP_CATEGORIES.id AS RELATIONSHIP, 
  RACE_CATEGORIES.id AS RACE, 
  SEX_CATEGORIES.id AS SEX, 
  NATIVE_COUNTRY_CATEGORIES.id AS NATIVE_COUNTRY, 
  CLASS_CATEGORIES.id AS CLASS 
FROM adult_with_id as adult
JOIN WORKCLASS_CATEGORIES ON adult.WORKCLASS = WORKCLASS_CATEGORIES.WORKCLASS 
JOIN EDUCATION_CATEGORIES ON adult.EDUCATION = EDUCATION_CATEGORIES.EDUCATION 
JOIN MARITAL_STATUS_CATEGORIES ON adult.MARITAL_STATUS = MARITAL_STATUS_CATEGORIES.MARITAL_STATUS 
JOIN OCCUPATION_CATEGORIES ON adult.OCCUPATION = OCCUPATION_CATEGORIES.OCCUPATION 
JOIN RELATIONSHIP_CATEGORIES ON adult.RELATIONSHIP = RELATIONSHIP_CATEGORIES.RELATIONSHIP 
JOIN RACE_CATEGORIES ON adult.RACE = RACE_CATEGORIES.RACE 
JOIN SEX_CATEGORIES ON adult.SEX = SEX_CATEGORIES.SEX 
JOIN NATIVE_COUNTRY_CATEGORIES ON adult.NATIVE_COUNTRY = NATIVE_COUNTRY_CATEGORIES.NATIVE_COUNTRY 
JOIN CLASS_CATEGORIES ON adult.CLASS = CLASS_CATEGORIES.CLASS
"""))
print("preprocessed count:",c.execute("select count(*) from ADULT_PREPROCESSED").fetchall())
print("preprocessed raw size in MB",c.execute("select RAW_OBJECT_SIZE/(1024*1024) from EXA_USER_OBJECT_SIZES where OBJECT_NAME='ADULT_PREPROCESSED'").fetchall())
c.export_to_pandas("select * from adult_preprocessed limit 10")

preprocessed count: [(1447104,)]
preprocessed raw size in MB [('276.01318359375',)]


,ID,AGE,FNLWGT,EDUCATION_NUM,CAPITALGAIN,CAPITALLOSS,HOURSPERWEEK,WORKCLASS,EDUCATION,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,NATIVE_COUNTRY,CLASS
0,295288,0.50,0.099751,0.666667,0,0,0.50,8,7,1,13,5,2,0,29,0
1,295289,1.00,0.131473,0.533333,0,0,0.50,8,4,6,8,1,2,0,29,1
2,295291,0.25,0.106148,0.600000,0,0,0.50,8,13,4,8,5,2,0,29,0
3,295297,1.00,0.138466,0.600000,0,0,0.50,7,13,1,10,5,2,0,29,0
4,295300,1.00,0.044264,0.533333,0,0,0.50,5,4,1,14,3,2,0,29,0
5,295303,0.50,0.013905,0.800000,0,0,0.75,8,14,1,10,5,2,0,29,0
6,295304,0.50,0.131596,0.600000,0,0,0.50,1,13,4,14,5,2,0,29,0
7,295306,0.00,0.138772,0.600000,0,0,0.50,7,13,4,10,3,2,0,29,0
8,295310,0.00,0.138703,0.533333,0,0,0.25,8,4,4,9,4,2,0,29,0
9,295315,0.75,0.105415,0.666667,0,0,0.50,8,7,1,14,3,2,0,29,0


In [5]:
import textwrap
c.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3 SCALAR SCRIPT udf_utils() RETURNS int AS

def reset(ctx):
  ctx.reset()
  df=ctx.get_dataframe(1,start_col=1)

def iterate_trough_dataset(ctx, batch_size, 
                          map_function,
                          init_function,
                          aggregate_function):
  reset(ctx)
  number_of_tuples_left=ctx.size()-1
  state = init_function()
  while True:
    if number_of_tuples_left<batch_size:
      if number_of_tuples_left>0:
        df = ctx.get_dataframe(number_of_tuples_left,start_col=1)
        number_of_tuples_left=0
      else:
        reset(ctx)
        break
    else:
      df = ctx.get_dataframe(batch_size,start_col=1)
      number_of_tuples_left=number_of_tuples_left-batch_size
    result = map_function(df)
    state = aggregate_function(state, result)
  return state
/
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3 SCALAR SCRIPT bucketfs_utils(...)
RETURNS INT AS
import os
import requests
import uuid
from sklearn.externals import joblib

def upload_object_to_bucketfs(object, host, bucket_name, path, user, password, secure=True):
    temp_file = "/tmp/" + str(uuid.uuid4().hex + ".pkl")
    joblib.dump(object, temp_file, compress=True)
    protocol = 'https' if secure else 'http'

    with open(temp_file, "rb") as f:
        url = f"{protocol}://{user}:{password}@{host}/{bucket_name}/{path}"
        response=requests.put(url, data=f)
        if response.status_code != 200:
            raise Exception(str(response)+" "+url)
    try:
        os.remove(temp_file)
    except OSError:
        pass

def download_object_from_bucketfs(host, bucket_name, path, user, password, secure=True):
    protocol = 'https' if secure else 'http'
    url = f"{protocol}://{user}:{password}@{host}/{bucket_name}/{path}"
    response=requests.get(url)
    if response.status_code != 200:
        raise Exception(str(response))
    temp_file = "/tmp/" + str(uuid.uuid4().hex + ".pkl")
    with open(temp_file, "wb") as f:
        f.write(response.content)
    object = joblib.load(temp_file)
    try:
        os.remove(temp_file)
    except OSError:
        pass
    return object
/
"""))

c.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3 SCALAR SCRIPT model_utils() RETURNS int AS
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder

udf_utils = exa.import_script('ML_TEST.UDF_UTILS')

def get_config(ctx):
  df=ctx.get_dataframe(1,start_col=2)
  inputs = df.iloc[:,0:-1]
  categorical_input_column_names=[column for column in inputs.columns if inputs[column].iloc[0]>0]
  numerical_input_column_names=[column for column in inputs.columns if inputs[column].iloc[0]==-1]
  input_column_categories = np.array([int(inputs[column].iloc[0])+1 for column in categorical_input_column_names])
  target_column_name = df.columns[-1]
  target_classes=int(df[target_column_name].iloc[0])+1
  return {"categorical_input_column_names": categorical_input_column_names, 
          "numerical_input_column_names": numerical_input_column_names, 
          "input_column_categories":input_column_categories, 
          "target_column_name":target_column_name, 
          "target_classes":target_classes}

def create_model(ctx,data_config):
  input_column_one_hot_encoder=create_one_hot_encoder(ctx,
    data_config["input_column_categories"],
    data_config["categorical_input_column_names"])
  classifier=SGDClassifier(loss='log',n_jobs=1, warm_start=True)
  return {"input_column_one_hot_encoder":input_column_one_hot_encoder,"classifier":classifier}

def create_one_hot_encoder(ctx,categories,categorical_column_names):
  one_hot_encoder = OneHotEncoder(n_values=categories, sparse=False)
  udf_utils.reset(ctx)
  data=ctx.get_dataframe(2,start_col=2)[categorical_column_names].astype(int).values
  one_hot_encoder.fit(data)
  return one_hot_encoder

/
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3 SCALAR SCRIPT preprocessing_utils() RETURNS int AS
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder

def encode_categorical_columns(categorical_columns: DataFrame, one_hot_encoder: OneHotEncoder):
  transformed_columns = DataFrame(one_hot_encoder.transform(categorical_columns.values))
  return transformed_columns

def preprocess_batch(df, data_config, 
                    input_column_one_hot_encoder):
  categorical_input_column_names = data_config["categorical_input_column_names"]
  numerical_input_column_names = data_config["numerical_input_column_names"]
  target_column_name = data_config["target_column_name"]

  encoded_input_categorical_columns = encode_categorical_columns(
                                          df[categorical_input_column_names],
                                          input_column_one_hot_encoder)
  numerical_input_columns = df[numerical_input_column_names].astype(float)
  input_columns = pd.concat([encoded_input_categorical_columns, 
                            numerical_input_columns],axis=1)
  target_column = df[target_column_name].astype(int)
  return input_columns, target_column


/
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3 SCALAR SCRIPT compute_score_utils() RETURNS int AS
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder

udf_utils = exa.import_script('ML_TEST.UDF_UTILS')
model_utils = exa.import_script('ML_TEST.MODEL_UTILS')
preprocessing_utils = exa.import_script('ML_TEST.PREPROCESSING_UTILS')

def compute_score(ctx,batch_size, data_config, model):
  final_state = udf_utils.iterate_trough_dataset(
    ctx,batch_size,
    lambda df: compute_score_batch(ctx, df, data_config, model),
    lambda: {"count":0,"score_sum":0},
    lambda state, result: {"count":state["count"]+result[0],"score_sum":state["score_sum"]+result[1]})
  score=final_state["score_sum"]/final_state["count"]
  return score

def compute_score_batch(ctx, df, data_config, model):
  input_column_one_hot_encoder = model["input_column_one_hot_encoder"]
  input_columns, target_column = preprocessing_utils.preprocess_batch(
                                                  df, data_config,
                                                  input_column_one_hot_encoder)
  classifier = model["classifier"]
  score=classifier.score(input_columns.values,target_column.values)
  return len(df),len(df)*score
/
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3 SCALAR SCRIPT predict_utils() RETURNS int AS
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder

udf_utils = exa.import_script('ML_TEST.UDF_UTILS')
model_utils = exa.import_script('ML_TEST.MODEL_UTILS')
preprocessing_utils = exa.import_script('ML_TEST.PREPROCESSING_UTILS')

def predict_batch(ctx, df, data_config, model):
  input_column_one_hot_encoder = model["input_column_one_hot_encoder"]
  input_columns, target_column = preprocessing_utils.preprocess_batch(
                                                  df, data_config,
                                                  input_column_one_hot_encoder)
  target_classes=np.arange(data_config["target_classes"])
  classifier = model["classifier"]
  result = classifier.predict(input_columns.values)
  return DataFrame({"0":df["1"],"1":target_column,"2":result,"3":df["1"].astype(str).apply(lambda x: "prediction")})

def predict(ctx,batch_size, data_config, model):
  udf_utils.iterate_trough_dataset(
    ctx,batch_size,
    lambda df: predict_batch(ctx, df, data_config, model),
    lambda: None,
    lambda state, result: ctx.emit( 
        result
      )
    )
/
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3 SCALAR SCRIPT train_utils() RETURNS int AS
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder

udf_utils = exa.import_script('ML_TEST.UDF_UTILS')
model_utils = exa.import_script('ML_TEST.MODEL_UTILS')
preprocessing_utils = exa.import_script('ML_TEST.PREPROCESSING_UTILS')

def train_batch(ctx, df, data_config, model):
  input_column_one_hot_encoder = model["input_column_one_hot_encoder"]
  input_columns, target_column = preprocessing_utils.preprocess_batch(
                                                  df, data_config,
                                                  input_column_one_hot_encoder)
  target_classes=np.arange(data_config["target_classes"])
  classifier = model["classifier"]
  classifier.partial_fit(input_columns.values,target_column.values, classes=target_classes)

def train(ctx,batch_size, data_config, model):
  udf_utils.iterate_trough_dataset(
    ctx,batch_size,
    lambda df: train_batch(ctx, df, data_config, model),
    lambda: None,
    lambda state, result: None)

/
"""))
c.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3 SET SCRIPT train_sgd_classifier(...) EMITS (result_column VARCHAR(2000000)) AS
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder

import uuid

model_utils = exa.import_script('ML_TEST.MODEL_UTILS')
bucketfs_utils = exa.import_script('ML_TEST.BUCKETFS_UTILS')
predict_utils = exa.import_script('ML_TEST.PREDICT_UTILS')
compute_score_utils = exa.import_script('ML_TEST.COMPUTE_SCORE_UTILS')
train_utils = exa.import_script('ML_TEST.TRAIN_UTILS')

def run(ctx):
  data_config = model_utils.get_config(ctx)
  model = model_utils.create_model(ctx,data_config) 
  epochs=5
  batch_size=1000
  base_path = f"models/{uuid.uuid4()}"
  ctx.emit(base_path)
  for i in range(epochs):
    train_utils.train(ctx, batch_size, data_config, model)
    score=compute_score_utils.compute_score(ctx, batch_size, data_config, model)
    ctx.emit(f"epoch:{i} score:{score}")
    if score==1.0:
      break
  model_path = f"{base_path}/model.joblib"
  data_config_path = f"{base_path}/data_config.joblib"
  bucketfs_utils.upload_object_to_bucketfs(model, "localhost:6583", "default", model_path, "w", "write", secure=False)
  bucketfs_utils.upload_object_to_bucketfs(data_config, "localhost:6583", "default", data_config_path, "w", "write", secure=False)
/
"""))

c.execute(textwrap.dedent("""
CREATE OR REPLACE PYTHON3 SET SCRIPT predict_sgd_classifier(...) EMITS (id int, class int, predicted_class int, comment VARCHAR(2000000)) AS
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder

bucketfs_utils = exa.import_script('ML_TEST.BUCKETFS_UTILS')
predict_utils = exa.import_script('ML_TEST.PREDICT_UTILS')
compute_score_utils = exa.import_script('ML_TEST.COMPUTE_SCORE_UTILS')

def init(ctx):
  df=ctx.get_dataframe(num_rows="all")
  ctx.reset()
  base_path=df['0'][0]
  batch_size=1000
  model_path = f"{base_path}/model.joblib"
  data_config_path = f"{base_path}/data_config.joblib"
  model=bucketfs_utils.download_object_from_bucketfs("localhost:6583", "default", model_path, "w", "write", secure=False)
  data_config=bucketfs_utils.download_object_from_bucketfs("localhost:6583", "default", data_config_path, "w", "write", secure=False)
  return model, data_config, batch_size

def run(ctx):
  model, data_config, batch_size = init(ctx)
  predict_utils.predict(ctx, batch_size, data_config, model)
/
"""))

<ExaStatement session_id=1671664255991602287 stmt_idx=37>

In [6]:
r=c.execute("""
select train_sgd_classifier(
                mode, id, age, fnlwgt, education_num, capitalgain, capitalloss, hoursperweek, 
                workclass, education, marital_status, occupation, relationship, race, sex, native_country, class
                order by mode desc nulls last)
from (
  select 'c'  as mode, null as id,
  -1 as age, -1 as fnlwgt, -1 as education_num, -1 as capitalgain, -1 as capitalloss, -1 hoursperweek, 
  (SELECT MAX(id) FROM WORKCLASS_CATEGORIES) as workclass,
  (SELECT MAX(id) FROM EDUCATION_CATEGORIES) as education,
  (SELECT MAX(id) FROM MARITAL_STATUS_CATEGORIES) as marital_status,
  (SELECT MAX(id) FROM OCCUPATION_CATEGORIES) as occupation,
  (SELECT MAX(id) FROM RELATIONSHIP_CATEGORIES) as relationship,
  (SELECT MAX(id) FROM RACE_CATEGORIES) as race,
  (SELECT MAX(id) FROM SEX_CATEGORIES) as sex,
  (SELECT MAX(id) FROM NATIVE_COUNTRY_CATEGORIES) as native_country,
  (SELECT MAX(id) FROM CLASS_CATEGORIES) as class
  union all
  select null as mode, id, age, fnlwgt, education_num, capitalgain, capitalloss, hoursperweek, 
          workclass, education, marital_status, occupation, relationship, race, sex, native_country, class
  from adult_preprocessed
  limit 100
) as q
""")
base_path=r.fetchone()[0]
print("base_path:",base_path)
for i,t in enumerate(r.fetchall()):
  print(t[0])
  if i>50:
    break


base_path: models/b3db178f-a873-4491-b751-dcf9ab8dddf6
epoch:0 score:0.9393939393939394
epoch:1 score:0.9595959595959596
epoch:2 score:0.9393939393939394
epoch:3 score:0.9696969696969697
epoch:4 score:0.9494949494949495


In [7]:
print(base_path)

models/b3db178f-a873-4491-b751-dcf9ab8dddf6


In [8]:
r=c.execute(f"""
select predict_sgd_classifier(
                mode, id, age, fnlwgt, education_num, capitalgain, capitalloss, hoursperweek, 
                workclass, education, marital_status, occupation, relationship, race, sex, native_country, class
                order by mode desc nulls last)
from (
  select '{base_path}' as mode, id, age, fnlwgt, education_num, capitalgain, capitalloss, hoursperweek, 
          workclass, education, marital_status, occupation, relationship, race, sex, native_country, class
  from adult_preprocessed
) as q
group by iproc(), mod(rownum,2)
""")
r.fetchall()

[(4, 0, 0, 'prediction'),
 (6, 0, 1, 'prediction'),
 (9, 1, 0, 'prediction'),
 (10, 1, 1, 'prediction'),
 (12, 1, 1, 'prediction'),
 (16, 0, 0, 'prediction'),
 (18, 0, 0, 'prediction'),
 (20, 1, 0, 'prediction'),
 (21, 1, 1, 'prediction'),
 (24, 0, 0, 'prediction'),
 (25, 0, 0, 'prediction'),
 (27, 0, 0, 'prediction'),
 (30, 0, 1, 'prediction'),
 (32, 0, 0, 'prediction'),
 (34, 0, 0, 'prediction'),
 (36, 0, 0, 'prediction'),
 (40, 0, 1, 'prediction'),
 (42, 0, 1, 'prediction'),
 (44, 0, 0, 'prediction'),
 (45, 0, 0, 'prediction'),
 (47, 0, 0, 'prediction'),
 (50, 0, 0, 'prediction'),
 (51, 0, 1, 'prediction'),
 (55, 0, 0, 'prediction'),
 (57, 0, 0, 'prediction'),
 (59, 0, 0, 'prediction'),
 (61, 0, 1, 'prediction'),
 (64, 1, 1, 'prediction'),
 (66, 0, 1, 'prediction'),
 (67, 0, 0, 'prediction'),
 (71, 0, 0, 'prediction'),
 (72, 0, 0, 'prediction'),
 (74, 0, 0, 'prediction'),
 (76, 0, 0, 'prediction'),
 (80, 0, 1, 'prediction'),
 (82, 0, 0, 'prediction'),
 (83, 0, 0, 'prediction'),
 (86

In [6]:
#r=c.execute("select count(*) from adult_preprocessed")
r=c.execute("select count(*) from (select 1 union all (select age from adult_preprocessed limit 10)) as q")
r.fetchall()

[(10,)]

  age integer,
  workclass varchar(2000000),
  fnlwgt integer,	
  education varchar(2000000),
  education_num integer,	
  marital_status varchar(2000000),
  occupation varchar(2000000),
  relationship varchar(2000000),
  race varchar(2000000),
  sex varchar(2000000),
  capitalgain integer,	
  capitalloss integer,	
  hoursperweek integer,
  native_country varchar(2000000),
  class varchar(2000000)